In [5]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import os
from scipy.stats import norm


In [6]:
 data_path = os.path.join("..", "data")

In [7]:
X = np.load(data_path + '/initial_data/function_2/initial_inputs.npy')
y = np.load(data_path + '/initial_data/function_2/initial_outputs.npy')
print(f"X is:{X}")
print(f"Y is :{y}")

X is:[[0.66579958 0.12396913]
 [0.87779099 0.7786275 ]
 [0.14269907 0.34900513]
 [0.84527543 0.71112027]
 [0.45464714 0.29045518]
 [0.57771284 0.77197318]
 [0.43816606 0.68501826]
 [0.34174959 0.02869772]
 [0.33864816 0.21386725]
 [0.70263656 0.9265642 ]]
Y is :[ 0.53899612  0.42058624 -0.06562362  0.29399291  0.21496451  0.02310555
  0.24461934  0.03874902 -0.01385762  0.61120522]


In [8]:
# add the week 1 input and result to the array
X = np.append(X, [[0.795918,0.959184 ]], axis=0)
y = np.append(y, [0.078279871], axis=0)

# add the week 2 input and result to the array
X = np.append(X, [[0.714286, 0.938776]], axis=0)
y = np.append(y, [0.5239853412950183], axis=0)

# add the week 3 input and result to the array
X = np.append(X, [[0.656566, 0.939394]], axis=0)
y = np.append(y, [0.414404463864519], axis=0)

# add the week 4 input and result to the array
X = np.append(X, [[0.729253, 0.058662]], axis=0)
y = np.append(y, [0.5073132554292137], axis=0)

# add the week 5 input and result to the array
X = np.append(X, [[0.999980, 0.502674]], axis=0)
y = np.append(y, [-0.007820208557515348], axis=0)

# add the week 6 input and result to the array
X = np.append(X, [[0.727273, 0.848485]], axis=0)
y = np.append(y, [0.5692408682454391], axis=0)

# add the week 7 input and result to the array
X = np.append(X, [[0.767677, 0.171717]], axis=0)
y = np.append(y, [0.19283512655684448], axis=0)

# add the week 8 input and result to the array
X = np.append(X, [[0.636364, 0.040404]], axis=0)
y = np.append(y, [0.14715688566886923], axis=0)

# add the week 9 input and result to the array
X = np.append(X, [[0.696970, 0.858586]], axis=0)
y = np.append(y, [0.6356443554454597], axis=0)

print(f"X is:{X}")
print(f"Y is :{y}")

X is:[[0.66579958 0.12396913]
 [0.87779099 0.7786275 ]
 [0.14269907 0.34900513]
 [0.84527543 0.71112027]
 [0.45464714 0.29045518]
 [0.57771284 0.77197318]
 [0.43816606 0.68501826]
 [0.34174959 0.02869772]
 [0.33864816 0.21386725]
 [0.70263656 0.9265642 ]
 [0.795918   0.959184  ]
 [0.714286   0.938776  ]
 [0.656566   0.939394  ]
 [0.729253   0.058662  ]
 [0.99998    0.502674  ]
 [0.727273   0.848485  ]
 [0.767677   0.171717  ]
 [0.636364   0.040404  ]
 [0.69697    0.858586  ]]
Y is :[ 0.53899612  0.42058624 -0.06562362  0.29399291  0.21496451  0.02310555
  0.24461934  0.03874902 -0.01385762  0.61120522  0.07827987  0.52398534
  0.41440446  0.50731326 -0.00782021  0.56924087  0.19283513  0.14715689
  0.63564436]


In [9]:
print(f"max y is :{np.max(y)}")

max y is :0.6356443554454597


In [10]:
# Define GP model
kernel = Matern(nu=2.5, length_scale_bounds=(1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, normalize_y=True)
gp.fit(X, y)

GaussianProcessRegressor(kernel=Matern(length_scale=1, nu=2.5),
                         normalize_y=True)

In [ ]:
# UCB acquisition function
# week 6 changed kappa to 4.0 from 2.0
# week 8 chnaged kappa to 3.0 from 4.0 for more exploitation
def ucb(x, gp, kappa=3.0):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return mu + kappa * sigma

In [11]:
# Expected Improvement (EI) acquisition function
def expected_improvement(x, gp, y_best, xi=0.01):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    
    if sigma == 0:
        return 0.0
    
    Z = (mu - y_best - xi) / sigma
    ei = (mu - y_best - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

In [12]:
grid_size = 100
x1 = np.linspace(0, 1, grid_size)
x2 = np.linspace(0, 1, grid_size)
X_grid = np.array([[a, b] for a in x1 for b in x2])

In [ ]:
ucb_values = np.array([ucb(x, gp) for x in X_grid])


In [13]:
# Compute EI values
y_best = np.max(y)
ei_values = np.array([expected_improvement(x, gp, y_best) for x in X_grid])

In [ ]:
best_idx = np.argmax(ucb_values)
next_best_point = X_grid[best_idx]
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)
print("Predicted maximum value (mean):", pred_mean[0])
print("Next best point to evaluate (UCB):", np.round(next_best_point,6))
print("Next best point to evaluate (UCB) without rounding:", next_best_point)
print("Predicted maximum value (mean):", pred_mean[0])

In [14]:
# Select next best point
best_idx = np.argmax(ei_values)
next_best_point = X_grid[best_idx]
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)

print("Next best point to evaluate (EI):", np.round(next_best_point, 6))
print("Next best point to evaluate (EI) without rounding:", next_best_point)
print("Predicted mean at next point:", pred_mean[0])
print("Expected Improvement value:", ei_values[best_idx])

Next best point to evaluate (EI): [0.707071 0.888889]
Next best point to evaluate (EI) without rounding: [0.70707071 0.88888889]
Predicted mean at next point: 0.6803318211703027
Expected Improvement value: 0.04900980272591039


In [ ]:
# Scale inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train SVM surrogate
svm_surrogate = SVR(kernel='rbf', C=10.0, epsilon=0.1)
svm_surrogate.fit(X_scaled, y)

# Predict on new candidates
X_new = np.random.rand(10000000, 2)
X_new_scaled = scaler.transform(X_new)
y_pred = svm_surrogate.predict(X_new_scaled)

# Maximization: find best candidate
best_idx = np.argmax(y_pred)
best_candidate = X_new[best_idx]
best_predicted_value = y_pred[best_idx]


print("Best candidate input:", best_candidate)
print("Best candidate input rounded:", np.round(best_candidate, 6))
print("Predicted maximum value:", best_predicted_value)